# Workbook Summary

This workbook analyses the statsitics from Tau and ABeta extracts, and formed part of the second WP of C319.  It is not intended for either demonstration or use with synthetic data, but will generate the reports found in the technical report given the segmentations produced by the other WP2 workbooks.

In [ ]:
%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
os.chdir("../..")
from polygeist.utils import get_case_and_slide, load_jsons_from_directory, load_filenames_and_generate_conditions

In [ ]:
config = {
    "dump_path": "/run/media/brad/ScratchM2/ABeta_label_dump_64/",
    "case_files": "Data/filenames/abeta_files.txt",
}

In [ ]:
case_conditions = load_filenames_and_generate_conditions(config["case_files"])

In [ ]:
# Labels used for each condition produced by load_filenames_and_generate_conditions
CONTROL = 'C'
AD = 'AD'
PD = 'PD'
MSA = 'MSA'
def slide_statistics_for_slide(jsons, slide_filter=None, norm=True):
    control = []
    ad = []
    pd = []
    msa = []
    for file in jsons.keys():
        case, slide = get_case_and_slide(file)
        if slide_filter:
            if slide != slide_filter:
                continue
        arr = np.array(jsons[file]['densities'])
        n = 1 if not norm else arr.shape[0] * arr.shape[1]
        h = jsons[file]['roi_count'] / n
        if case in case_conditions:
            condition = case_conditions[case]
            if condition == CONTROL:
                control.append(h)
            elif condition == AD:
                ad.append(h)
            elif condition == PD:
                pd.append(h)
            else:
                msa.append(h)
        else:
            print(f"Case not found {file}")
    return control, ad, pd, msa

In [ ]:
def slide_statistics_to_csv(jsons, filename):
    # Create a list to be writeen to file
    csv_table = ["case, score, slide, condition\n"]
    conditon_map = {
        "C" : 0,
        "PD": 1,
        "AD": 2,
        "MSA":3,
    }
    for file in jsons.keys():
        case, slide = get_case_and_slide(file)
        condition = case_conditions[case]
        row = f"{case}, {jsons[file]['roi_count']}, {slide}, {conditon_map[condition]}\n"
        csv_table.append(row)
    
    with open(filename, 'w') as f:
        for r in csv_table:
            f.write(r)

In [ ]:
jsons = load_jsons_from_directory(config['dump_path'])

In [ ]:
norm_rois = True
control, ad, pd, msa = slide_statistics_for_slide(jsons, slide_filter=None, norm=norm_rois)
fig, ax = plt.subplots()
ax.boxplot([control, pd, ad], showfliers=False)
ax.set_xticks([1, 2, 3], ['Control', 'PD', 'AD+PD'])
set_labels(ax, "Pathology Condition", f"Quartiles of ROI {'Counts' if not norm_rois else 'Proportions'}", "")
plt.show()

In [ ]:
slides_abeta = (1, 3, 4, 5, 7, 9, 10, 12, 14, 15)

In [ ]:
# subplots
fig, axs = plt.subplots(2, 5, sharey=True)
for i, ind in enumerate(slides_abeta):
    control, ad, pd, msa = slide_statistics_for_slide(jsons, slide_filter=ind, norm=norm_rois)
    ax = axs[int(i / 5)][i % 5]
    ax.boxplot([control, pd, ad], showfliers=False)
    ax.set_xticks([1, 2, 3], ['Control', 'PD', 'AD+PD'])
    ax.set_title(f"Slide {ind}")
plt.savefig("pathology_by_slide.svg")
plt.show()

In [ ]:
# Write the CSV out to do stats in JASP.
slide_statistics_to_csv(jsons, 'abeta_counts.csv')

In [ ]:
## TAU
config = {
    "dump_path": "/run/media/brad/ScratchM2/Tau_dump_256_background_removed",
    "case_files": "Data/filenames/tau_files.txt",
}

In [ ]:
case_conditions = load_filenames_and_generate_conditions(config["case_files"])

In [ ]:
jsons = load_jsons_from_directory(config['dump_path'])

In [ ]:
norm_rois = True
control, ad, pd, _ = slide_statistics_for_slide(jsons, slide_filter=None, norm=norm_rois)
fig, ax = plt.subplots()
ax.boxplot([control, pd, ad], showfliers=False)
ax.set_xticks([1, 2, 3], ['Control', 'PD', 'AD+PD'])
set_labels(ax, "Pathology Condition", f"Quartiles of ROI {'Counts' if not norm_rois else 'Proportions'}", "")
plt.show()

In [ ]:
slides_tau = (1, 4, 5, 9, 10, 12, 13, 15, 16)

In [ ]:
# subplots
fig, axs = plt.subplots(2, 5, sharey=True)
for i, ind in enumerate(slides_tau):
    control, ad, pd, msa = slide_statistics_for_slide(jsons, slide_filter=ind, norm=norm_rois)
    ax = axs[int(i / 5)][i % 5]
    ax.boxplot([control, pd, ad], showfliers=False)
    ax.set_xticks([1, 2, 3], ['Control', 'PD', 'AD+PD'])
    ax.set_title(f"Slide {ind}")
plt.show()

In [ ]:
# Write the CSV out to do stats in JASP.
slide_statistics_to_csv(jsons, 'tau_counts.csv')